In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data=df.copy()
df.head()

In [ ]:
data.info()

In [ ]:
#There's no null data

In [ ]:
y=data.Outcome
x=data.drop(['Outcome'], axis=1)

In [ ]:
#Let Visualize the Outcome
_=sns.catplot(x='Outcome', kind='count', data=data)
(data.Outcome).value_counts()


#Data healthly has 500 data and 268 are diabetes.

In [ ]:
x.describe().T

In [ ]:
#Standarization
data_diag=y
data=x

data_s=(data-data.mean())/(data.std())
data=pd.concat([y,data_s],axis=1)
data=pd.melt(data, id_vars='Outcome',var_name='features', value_name='value')

plt.figure(figsize=(20,10))
_=sns.violinplot(x='features',y='value', hue='Outcome', split=True,inner='quart',data=data)
plt.title('Violin Plot')


In [ ]:
#Standarization
data_diag=y
data=x

data_s=(data-data.mean())/(data.std())
data=pd.concat([y,data_s],axis=1)
data=pd.melt(data, id_vars='Outcome',var_name='features', value_name='value')

plt.figure(figsize=(20,10))
_=sns.boxplot(x='features',y='value', hue='Outcome', data=data)
plt.title('Box Plot')


In [ ]:
#Does Glucose and Insulin Correlated?
sns.jointplot(x.loc[:,'Glucose'], x.loc[:,'BMI'],kind='regg',color="#ce1414")

In [ ]:
corr=df[['Glucose','BMI']]
corr.corr()
sns.heatmap(corr.corr(),annot=True)

#It has low correlation

In [ ]:
g=sns.PairGrid(x.loc[:,["Glucose","BMI","Insulin","Age"]],diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot)


In [ ]:
#Standarization
data_diag=y
data=x

data_s=(data-data.mean())/(data.std())
data=pd.concat([y,data_s],axis=1)
data=pd.melt(data, id_vars='Outcome',var_name='features', value_name='value')

plt.figure(figsize=(20,10))
_=sns.swarmplot(x='features',y='value', hue='Outcome',data=data)
plt.title('Swarm Plot')


In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(df.corr(),annot=True)

In [ ]:
#Classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X=df.drop(['Outcome'],axis=1)
Y=df['Outcome']

X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2, random_state=31)

clf=RandomForestRegressor()
model=clf.fit(X_train, Y_train)

Y_pred=model.predict(X_test)



In [ ]:
Y_prednew=np.round(Y_pred, decimals=0)
Y_prednew

In [ ]:
print("Accuracy Score is : ", accuracy_score(Y_prednew, Y_test))
_=sns.heatmap(confusion_matrix(Y_prednew, Y_test), annot=True)

In [ ]:
#Feature Selection

from sklearn.feature_selection import SelectKBest, chi2
select_feature=SelectKBest(chi2, k=5).fit(X_train, Y_train)

print("Score List:",select_feature.scores_)
print("Feature list", X_train.columns)

#We will drop the bloodpressure and Deiabetes Pedigree Function


In [ ]:
#Classification


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X=data.drop(['Outcome','BloodPressure','DiabetesPedigreeFunction'],axis=1)
Y=data['Outcome']

X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2, random_state=31)

clf=RandomForestRegressor()
model=clf.fit(X_train, Y_train)

Y_pred=model.predict(X_test)
Y_prednew=np.round(Y_pred, decimals=0)
Y_prednew

In [ ]:
print("Accuracy Score is : ", accuracy_score(Y_prednew, Y_test))
_=sns.heatmap(confusion_matrix(Y_prednew, Y_test), annot=True)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()      
rfe = RFE(estimator=clf_rf_3, n_features_to_select=5, step=1)
rfe = rfe.fit(X_train, Y_train)
print('Chosen best 5 feature by rfe:',X_train.columns[rfe.support_])

Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct classifications
clf_rf_4 = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(X_train, Y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', X_train.columns[rfecv.support_])

In [ ]:
# Plot number of features VS. cross-validation scores
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()